# Import

## Library

In [2]:
import pandas as pd
import xlwt
import requests
import lxml.html as html
import time
import warnings
warnings.filterwarnings('ignore')

## Dataset

In [73]:
selection_word = ['great']
selection_word = pd.DataFrame( selection_word, columns=['word'] )
selection_word

,word
0,great


# Scrapy

## Functions

### Scrapy Promt

In [99]:
def scrapy_promr( uniqueword ):
    
    url = f'https://www.online-translator.com/contexts/english-spanish/{uniqueword}'
    df = pd.DataFrame(columns=['server','search_word','english_word','english_phrase','spanish_word','spanish_phrase','link'])
    
    
#     print(uniqueword,url)

    try:
        response = requests.get(url)
        if response.status_code == 200 :
            response2 = response.content.decode('utf-8')
            response3 = html.fromstring(response2)   
            for i in range(1,6):
                body_english = response3.xpath(f'//div[@class="allSamples"]/div[{i}]/span[@class="samSource"]//text()')
                word_english = response3.xpath(f'//div[@class="allSamples"]/div[{i}]/span[@class="samSource"]/span//text()')
                if len(word_english) != 0 :
                    word_english = word_english[0].lower()

                    body_english = ''.join(body_english).replace( f'{word_english}',f'[{word_english}]')
                    body_english = f'[{word_english}]  ' + body_english
                    body_english = body_english.lower()

                    body_spanish = response3.xpath(f'//div[@class="allSamples"]/div[{i}]/span[@class="samTranslation"]//text()')
                    body_spanish = ''.join(body_spanish)
                    body_spanish = body_spanish.lower()
                    word_spanish = response3.xpath(f'//div[@class="allSamples"]/div[{i}]/span[@class="samTranslation"]/a//text()')
                    word_spanish = ''.join(word_spanish).lower()

                    body_spanish = body_spanish.lower().replace( f'{word_spanish}',f'[{word_spanish}]')

                    df = df.append({'server' : 'Promt',
                                    'search_word' : uniqueword,
                                    'english_word': word_english,
                                    'english_phrase':body_english,
                                    'spanish_word':word_spanish ,
                                    'spanish_phrase':f'[ {word_spanish} ]  ' + body_spanish,
                                    'link':url},
                                   ignore_index=True)
                
        else:
            raise ValueError(f'Error : {response.status_code}')
            print('NO GOOD')
    except ValueError as ve:
        print(ve)
    
    return df 

### Linguee

In [93]:
def scrapy_linguee( uniqueword ):
    
    url = f'https://www.linguee.com/english-spanish/search?source=auto&query={uniqueword}'
    
    df = pd.DataFrame(columns=['server','search_word','english_word','english_phrase','spanish_word','spanish_phrase','link'])
    
#     print(uniqueword,url)

    try:
        response = requests.get(url)
        if response.status_code == 200 :

            response2 = response.content.decode('latin-1')
            response3 = html.fromstring(response2) 
            
            for i in range( 20) :  
                #spanish phase                 
                spanish_phrase = response3.xpath( f'//div[@class="exact"]/child::node()[1]//div[@class="translation_lines"]/child::node()[{i}]//div[@class="example_lines"]/child::node()[1]//span[@class="tag_t"]/text()' )
                spanish_word   = response3.xpath( f'//div[@class="exact"]/child::node()[1]//div[@class="translation_lines"]/child::node()[{i}]//span[@class="tag_trans"]//text()' )
                
                english_phrase = response3.xpath( f'//div[@class="exact"]/child::node()[1]//div[@class="translation_lines"]/child::node()[{i}]//div[@class="example_lines"]/child::node()[1]//span[@class="tag_s"]/text()')
                

                if (len(english_phrase) and len(spanish_word))  != 0 :  
#                         english_phrase = f'[{uniqueword}]' + english_phrase[0]

                        df = df.append({'server' : 'linguee',
                                        'search_word':uniqueword,
                                        'english_word':uniqueword,
                                        'english_phrase':f'[ {uniqueword} ]  ' + english_phrase[0].replace( f'{uniqueword}',f'[ {uniqueword} ]').lower(),
                                        'spanish_word' : spanish_word[0] ,
                                        'spanish_phrase': f'[ {spanish_word[0]} ]  ' + spanish_phrase[0].replace( f'{spanish_word[0]}',f'[ {spanish_word[0]} ]').lower(),                    
                                        'link':url }, ignore_index=True)

        else:
            raise ValueError(f'Error : {response.status_code}')
            print('NO GOOD')
    except ValueError as ve:
        print(ve)
    
    return df 



In [105]:
scrapy_linguee( 'run' )

Error : 503


,server,search_word,english_word,english_phrase,spanish_word,spanish_phrase,link


## Main

In [108]:
selection_word = ['run']
selection_word = pd.DataFrame( selection_word, columns=['word'] )


if __name__=='__main__':
    
    df_mother = pd.DataFrame(columns=['server','search_word','english_word','english_phrase','spanish_word','spanish_phrase','link'])

    for i in range(1):
        # PROMT.One
        df_aux_promr = scrapy_promr( selection_word['word'][i]  )
        df_mother = pd.concat([df_mother, df_aux_promr],axis=0)
        
#         word reference         
        df_aux_linguee = scrapy_linguee( selection_word['word'][i]  )
        df_mother = pd.concat([df_mother, df_aux_linguee],axis=0)
        
        #Reset index         
        df_mother = df_mother.reset_index(drop=True)

df_mother

,server,search_word,english_word,english_phrase,spanish_word,spanish_phrase,link
0,Promt,run,run,[run] take the money and [run].,corre,[ corre ] toma el dinero y [corre].,https://www.online-translator.com/contexts/eng...
1,Promt,run,run,[run] why did you [run] away?,huiste,[ huiste ] ¿por qué [huiste]?,https://www.online-translator.com/contexts/eng...
2,Promt,run,run,[run] i hardly ever [run] into him.,encuentro,[ encuentro ] yo rara vez me [encuentro] con él.,https://www.online-translator.com/contexts/eng...
3,Promt,run,run,[run] he managed to [run] the machine.,funcionara,[ funcionara ] consiguió que la máquina [func...,https://www.online-translator.com/contexts/eng...
4,Promt,run,run,[run] the old man was [run] over and immediat...,llevado,[ llevado ] el anciano fue atropellado y fue ...,https://www.online-translator.com/contexts/eng...
5,linguee,run,run,[ run ] cheetahs can [ run ] very fast.,correr,[ correr ] los guepardos pueden [ correr ] mu...,https://www.linguee.com/english-spanish/search...
6,linguee,run,run,[ run ] my company [ run ]s a sales contest e...,organizar,[ organizar ] mi empresa organiza un concurso...,https://www.linguee.com/english-spanish/search...
7,linguee,run,run,[ run ] my dream is to [ run ] my own business.,tener,[ tener ] mi sueño es [ tener ] mi propio neg...,https://www.linguee.com/english-spanish/search...
8,linguee,run,run,[ run ] this software can [ run ] several tas...,ejecutar,[ ejecutar ] este software puede [ ejecutar ]...,https://www.linguee.com/english-spanish/search...
9,linguee,run,run,[ run ] we appointed a new manager to [ run ]...,dirigir,[ dirigir ] nombramos a un nuevo gerente para...,https://www.linguee.com/english-spanish/search...
